In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from dataset_loader import get_dataloaders, visualize_batch, visualize_batch_pred, visualize_uv_flow
from model import ResNetUnet, MaskedL1Loss, MaskedMSELoss, SSIMLoss, UVReconstructionLoss
from training_val import validate


import torch
from typing import Dict, List, Tuple, Optional, Callable
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np

In [ ]:
# Configuration
DATA_DIR = '/home/daniel-choate/Datasets/DocUnwarp/renders/synthetic_data_pitch_sweep'
BATCH_SIZE = 8
IMG_SIZE = (256, 256)
WEIGHTS_PATH = 'best_model_256.pth'
NUM_VISUALS = 5  # number of validation samples to visualize

# LOAD DATA
_, val_loader = get_dataloaders(
    data_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    use_depth=False,
    use_uv=True,
    use_border=True
)

# Load device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Load Model 
model = ResNetUnet(
    backbone_name = 'resnet34',
    pretrained = True
    ).to(device)
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=device))

# Loss function (ALTER IF NEEDED)
# criterion = nn.MSELoss()
# criterion = SSIMLoss()
criterion = UVReconstructionLoss(
        reconstruction_weight=1.0,
        uv_weight=0.5,
        smoothness_weight=0.01, 
        use_mask=True,
        loss_type='ssim' # Use SSIM for geometric recon
    )

# Validation loss
val_loss = validate(model, val_loader, criterion, device)
print(f"Validation loss: {val_loss:.4f}")


# VISUALIZE PREDICTIONS
model.eval()
visualized = 0
with torch.no_grad():
    for batch in val_loader:
        rgb = batch['rgb'].to(device)
        gt = batch['ground_truth'].to(device)
        uv_gt = batch['uv'].to(device)
        
        # Forward pass
        # output = model(rgb)
        outputs = model(rgb, predict_uv=True)
        warped = outputs['warped']
        flow = outputs['flow']
        uv_pred = outputs['uv']

        # --- STEP 1: Print UV value ranges ---
        print("Predicted UV: min =", uv_pred.min().item(), "max =", uv_pred.max().item())
        print("GT UV: min =", uv_gt.min().item(), "max =", uv_gt.max().item())
        print("UV shapes: pred =", uv_pred.shape, ", gt =", uv_gt.shape)

        for i in range(rgb.size(0)):
            if visualized >= NUM_VISUALS:
                break

            visualize_batch_pred({
                'rgb': rgb[i].unsqueeze(0).cpu(),
                'ground_truth': gt[i].unsqueeze(0).cpu(),
                'predicted': warped[i].unsqueeze(0).cpu(),
            }, num_samples=1)

            # ---- UV FLOW FIELD ----
            visualize_uv_flow(
                uv_pred[i].unsqueeze(0).cpu(),  # shape [1,2,H,W]
                uv_gt[i].unsqueeze(0).cpu(),
                num_samples=1,
                title_pred="Predicted UV Flow",
                title_gt = "Ground truth UV Flow"
            )

            visualized += 1

        if visualized >= NUM_VISUALS:
            break

In [ ]:
# Loss plot 
import matplotlib.pyplot as plt
import torch

history = torch.load('training_history_256.pth')

train_losses = history['train_losses']
val_losses = history['val_losses']
epochs = range(1, 50 + 1)

plt.figure(figsize=(8, 5))
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Validation Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss Curve')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()